In [5]:
import pandas as pd
import threading
import time
import openai
from threading import Lock
from langchain_community.embeddings import OpenAIEmbeddings
# from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()
import json
from langchain_community.vectorstores import Chroma
import pandas as pd
import sys
import ast

In [6]:
df = pd.read_csv("FinalCTTrialsDF_Full_w_ContactInfo_LocList_testing.csv")

In [9]:
df

,Unnamed: 0,NCT_NUMBER,STUDY_TITLE,STUDY_URL,STUDY_STATUS,CONDITIONS,LOCATIONS,START_DATE,COMPLETION_DATE,AGE,ELIGIBILITY_CRITERIA,PRIMARY_OUTCOMES,SECONDARY_OUTCOMES,INTERVENTIONS,POINT_OF_CONTACT
0,0,NCT05664971,JS004 Combined With Toripalimab and With Stand...,https://www.clinicaltrials.gov/study/NCT05664971,Recruiting,Advanced Lung Cancer,[{'Location Facility': 'Shanghai Chest Hospita...,"February 9, 2023","November 17, 2024",18 Years||Adult|Older Adult,Eligible participants include males or females...,The primary outcomes include the incidence of ...,Secondary outcomes encompass the duration of r...,The study involves multiple cohorts receiving ...,"{'Name': None, 'Organization': None, 'Email': ..."
1,1,NCT05902520,Adoptive Cell Therapy Using Cancer Specific CD...,https://www.clinicaltrials.gov/study/NCT05902520,Recruiting,"HNSCC, Melanoma, Gynecologic Cancer, Colorecta...",[{'Location Facility': 'Providence Portland Me...,"June 19, 2023","May 19, 2026",18 Years||Adult|Older Adult,Eligibility includes adults over 18 with histo...,The primary outcome is to assess the safety of...,Secondary outcomes include evaluating the effi...,The study involves two experimental arms. Both...,"{'Name': None, 'Organization': None, 'Email': ..."
2,2,NCT03917043,APG-2449 in Patients With Advanced Solid Tumors,https://www.clinicaltrials.gov/study/NCT03917043,Recruiting,"Advanced Solid Cancer, Non Small Cell Lung Can...",[{'Location Facility': 'Beijing Cancer Hospita...,"May 27, 2019",February 2025,18 Years||Adult|Older Adult,Eligibility includes patients with advanced tu...,The primary outcomes are to determine the maxi...,Secondary outcomes include measuring the maxim...,The intervention involves administering APG-24...,"{'Name': None, 'Organization': None, 'Email': ..."
3,3,NCT05674422,GEMCAD-REVEAL STUDY - Circulating Tumor DNA as...,https://www.clinicaltrials.gov/study/NCT05674422,Recruiting,Patients With Locally Advanced Rectal Cancer,[{'Location Facility': 'Hospital Germans Trias...,"May 26, 2023",July 2026,18 Years||Adult|Older Adult,Eligible participants are legally capable indi...,The primary outcome measure is the detection o...,Secondary outcomes include the classification ...,The intervention involves total neoadjuvant th...,"{'Name': None, 'Organization': None, 'Email': ..."
4,4,NCT06249854,Efficacy and Safety of Combination Therapy Wit...,https://www.clinicaltrials.gov/study/NCT06249854,Recruiting,Non-small Cell Lung Cancer,[{'Location Facility': 'Korea University Ansan...,"February 8, 2024","December 31, 2025",19 Years||Adult|Older Adult,Eligibility includes adults aged 19 or older w...,The primary outcome measure is Progression-Fre...,Secondary outcomes include Disease Control Rat...,The study involves two groups: one receiving P...,"{'Name': None, 'Organization': None, 'Email': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3831,3831,NCT05006131,Pancreatic Cancer Screening for At-risk Indivi...,https://www.clinicaltrials.gov/study/NCT05006131,Recruiting,"Pancreatic Cancer, Adult",[{'Location Facility': 'Beth Israel Deaconess ...,"July 10, 2020","July 10, 2026",18 Years|90 Years|Adult|Older Adult,Eligibility includes individuals indicated for...,The primary outcome is the proportion of scree...,Secondary outcomes include the proportion of s...,The intervention involves screening for pancre...,"{'Name': None, 'Organization': None, 'Email': ..."
3832,3832,NCT04679064,Trial on NIraparib-TSR-042 (Dostarlimab) vs Ph...,https://www.clinicaltrials.gov/study/NCT04679064,Recruiting,Ovarian Cancer,[{'Location Facility': 'Azienda Ospedaliera Sp...,"December 1, 2020","January 1, 2025",18 Years||Adult|Older Adult,"Eligibility includes recurrent ovarian, Fallop...",The primary outcome measure is overall surviva...,Secondary outcomes include progression-free su...,Interventions include a choice of standard che...,"{'Name': None, 'Organization': None, 'Email': ..."
3833,3833,NCT04491864,DWIBS-MRI: An Adjunct to the Trad

In [7]:
drugs_biomarkers_template=f'''
    Please extract all the names of the drugs and biomarkers from the information provided
    ---BEGIN FORMAT TEMPLATE---
{{"DRUGS":"list of name of the drugs"
"BIOMARKERS":"list of name of the biomarker"}}
---END FORMAT TEMPLATE---
Give the output of the format template in json format
    '''

In [10]:
def get_drugs_biomarkers(user_prompt):
        
        system_prompt=f'''
    Please extract all the names of the drugs and biomarkers from the information provided
    ---BEGIN FORMAT TEMPLATE---
{{"DRUGS":"list of name of the drugs","BIOMARKERS":"list of name of the biomarker"}}
---END FORMAT TEMPLATE---
Give the output of the format template in json format
    '''
        
        
        
        
        response = openai.chat.completions.create(
            model='gpt-4-0125-preview',
            temperature=0,
            messages=[
                {"role":"system", "content":system_prompt},
                {"role":"user", "content":str(user_prompt)},
            ],
            max_tokens = 1024,
            response_format={ "type": "json_object" }
            
        )
        res = response.choices[0].message.content
        return res


In [11]:
df["drugs & biomarker"]=df["INTERVENTIONS"].apply(get_drugs_biomarkers)

In [12]:
df

,Unnamed: 0,NCT_NUMBER,STUDY_TITLE,STUDY_URL,STUDY_STATUS,CONDITIONS,LOCATIONS,START_DATE,COMPLETION_DATE,AGE,ELIGIBILITY_CRITERIA,PRIMARY_OUTCOMES,SECONDARY_OUTCOMES,INTERVENTIONS,POINT_OF_CONTACT,drugs & biomarker
0,0,NCT05664971,JS004 Combined With Toripalimab and With Stand...,https://www.clinicaltrials.gov/study/NCT05664971,Recruiting,Advanced Lung Cancer,[{'Location Facility': 'Shanghai Chest Hospita...,"February 9, 2023","November 17, 2024",18 Years||Adult|Older Adult,Eligible participants include males or females...,The primary outcomes include the incidence of ...,Secondary outcomes encompass the duration of r...,The study involves multiple cohorts receiving ...,"{'Name': None, 'Organization': None, 'Email': ...","{""DRUGS"":[""JS004"",""JS001"",""docetaxel"",""pemetre..."
1,1,NCT05902520,Adoptive Cell Therapy Using Cancer Specific CD...,https://www.clinicaltrials.gov/study/NCT05902520,Recruiting,"HNSCC, Melanoma, Gynecologic Cancer, Colorecta...",[{'Location Facility': 'Providence Portland Me...,"June 19, 2023","May 19, 2026",18 Years||Adult|Older Adult,Eligibility includes adults over 18 with histo...,The primary outcome is to assess the safety of...,Secondary outcomes include evaluating the effi...,The study involves two experimental arms. Both...,"{'Name': None, 'Organization': None, 'Email': ...","{\n""DRUGS"":[""IL-2""],\n""BIOMARKERS"":[""CD39"", ""C..."
2,2,NCT03917043,APG-2449 in Patients With Advanced Solid Tumors,https://www.clinicaltrials.gov/study/NCT03917043,Recruiting,"Advanced Solid Cancer, Non Small Cell Lung Can...",[{'Location Facility': 'Beijing Cancer Hospita...,"May 27, 2019",February 2025,18 Years||Adult|Older Adult,Eligibility includes patients with advanced tu...,The primary outcomes are to determine the maxi...,Secondary outcomes include measuring the maxim...,The intervention involves administering APG-24...,"{'Name': None, 'Organization': None, 'Email': ...","{""DRUGS"":[""APG-2449""],""BIOMARKERS"":[]}"
3,3,NCT05674422,GEMCAD-REVEAL STUDY - Circulating Tumor DNA as...,https://www.clinicaltrials.gov/study/NCT05674422,Recruiting,Patients With Locally Advanced Rectal Cancer,[{'Location Facility': 'Hospital Germans Trias...,"May 26, 2023",July 2026,18 Years||Adult|Older Adult,Eligible participants are legally capable indi...,The primary outcome measure is the detection o...,Secondary outcomes include the classification ...,The intervention involves total neoadjuvant th...,"{'Name': None, 'Organization': None, 'Email': ...","{""DRUGS"":[],""BIOMARKERS"":[]}"
4,4,NCT06249854,Efficacy and Safety of Combination Therapy Wit...,https://www.clinicaltrials.gov/study/NCT06249854,Recruiting,Non-small Cell Lung Cancer,[{'Location Facility': 'Korea University Ansan...,"February 8, 2024","December 31, 2025",19 Years||Adult|Older Adult,Eligibility includes adults aged 19 or older w...,The primary outcome measure is Progression-Fre...,Secondary outcomes include Disease Control Rat...,The study involves two groups: one receiving P...,"{'Name': None, 'Organization': None, 'Email': ...","{\n""DRUGS"":[""Pembrolizumab"", ""Bojungikgitang""]..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3831,3831,NCT05006131,Pancreatic Cancer Screening for At-risk Indivi...,https://www.clinicaltrials.gov/study/NCT05006131,Recruiting,"Pancreatic Cancer, Adult",[{'Location Facility': 'Beth Israel Deaconess ...,"July 10, 2020","July 10, 2026",18 Years|90 Years|Adult|Older Adult,Eligibility includes individuals indicated for...,The primary outcome is the proportion of scree...,Secondary outcomes include the proportion of s...,The intervention involves screening for pancre...,"{'Name': None, 'Organization': None, 'Email': ...","{""DRUGS"":[],""BIOMARKERS"":[""CAPS3""]}"
3832,3832,NCT04679064,Trial on NIraparib-TSR-042 (Dostarlimab) vs Ph...,https://www.clinicaltrials.gov/study/NCT04679064,Recruiting,Ovarian Cancer,[{'Location Facility': 'Azienda Ospedaliera Sp...,"December 1, 2020","January 1, 2025",18 Years|

In [13]:
df.to_csv('finalcttrial.csv',index=False)

In [14]:
df["drugs & biomarker"]

0       {"DRUGS":["JS004","JS001","docetaxel","pemetre...
1       {\n"DRUGS":["IL-2"],\n"BIOMARKERS":["CD39", "C...
2                  {"DRUGS":["APG-2449"],"BIOMARKERS":[]}
3                            {"DRUGS":[],"BIOMARKERS":[]}
4       {\n"DRUGS":["Pembrolizumab", "Bojungikgitang"]...
                              ...                        
3831                  {"DRUGS":[],"BIOMARKERS":["CAPS3"]}
3832    {"DRUGS":["Pegylated Liposomal Doxorubicin", "...
3833    {\n"DRUGS":[],\n"BIOMARKERS":["MRI DWIBS", "DW...
3834                   {"DRUGS":[],"BIOMARKERS":["HER2"]}
3835    {"DRUGS":["sulforaphane","placebo"],"BIOMARKER...
Name: drugs & biomarker, Length: 3836, dtype: object

In [15]:
def parse_dict(text):
    try:
        return ast.literal_eval(text)
    except ValueError:
        return {}

In [16]:
df["DRUGS"] = df["drugs & biomarker"].apply(lambda x: parse_dict(x).get("DRUGS", []))
df["BIOMARKERS"] = df["drugs & biomarker"].apply(lambda x: parse_dict(x).get("BIOMARKERS", []))

In [17]:
df["DRUGS"]

0       [JS004, JS001, docetaxel, pemetrexed, carbopla...
1                                                  [IL-2]
2                                              [APG-2449]
3                                                      []
4                         [Pembrolizumab, Bojungikgitang]
                              ...                        
3831                                                   []
3832    [Pegylated Liposomal Doxorubicin, Weekly Pacli...
3833                                                   []
3834                                                   []
3835                              [sulforaphane, placebo]
Name: DRUGS, Length: 3836, dtype: object

In [18]:
df["DRUGS"] = df["DRUGS"].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

In [19]:
df["DRUGS"] 

0       JS004, JS001, docetaxel, pemetrexed, carboplat...
1                                                    IL-2
2                                                APG-2449
3                                                        
4                           Pembrolizumab, Bojungikgitang
                              ...                        
3831                                                     
3832    Pegylated Liposomal Doxorubicin, Weekly Paclit...
3833                                                     
3834                                                     
3835                                sulforaphane, placebo
Name: DRUGS, Length: 3836, dtype: object

In [20]:
df["BIOMARKERS"] = df["BIOMARKERS"].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

In [21]:
df

,Unnamed: 0,NCT_NUMBER,STUDY_TITLE,STUDY_URL,STUDY_STATUS,CONDITIONS,LOCATIONS,START_DATE,COMPLETION_DATE,AGE,ELIGIBILITY_CRITERIA,PRIMARY_OUTCOMES,SECONDARY_OUTCOMES,INTERVENTIONS,POINT_OF_CONTACT,drugs & biomarker,DRUGS,BIOMARKERS
0,0,NCT05664971,JS004 Combined With Toripalimab and With Stand...,https://www.clinicaltrials.gov/study/NCT05664971,Recruiting,Advanced Lung Cancer,[{'Location Facility': 'Shanghai Chest Hospita...,"February 9, 2023","November 17, 2024",18 Years||Adult|Older Adult,Eligible participants include males or females...,The primary outcomes include the incidence of ...,Secondary outcomes encompass the duration of r...,The study involves multiple cohorts receiving ...,"{'Name': None, 'Organization': None, 'Email': ...","{""DRUGS"":[""JS004"",""JS001"",""docetaxel"",""pemetre...","JS004, JS001, docetaxel, pemetrexed, carboplat...",
1,1,NCT05902520,Adoptive Cell Therapy Using Cancer Specific CD...,https://www.clinicaltrials.gov/study/NCT05902520,Recruiting,"HNSCC, Melanoma, Gynecologic Cancer, Colorecta...",[{'Location Facility': 'Providence Portland Me...,"June 19, 2023","May 19, 2026",18 Years||Adult|Older Adult,Eligibility includes adults over 18 with histo...,The primary outcome is to assess the safety of...,Secondary outcomes include evaluating the effi...,The study involves two experimental arms. Both...,"{'Name': None, 'Organization': None, 'Email': ...","{\n""DRUGS"":[""IL-2""],\n""BIOMARKERS"":[""CD39"", ""C...",IL-2,"CD39, CD103, PD-1"
2,2,NCT03917043,APG-2449 in Patients With Advanced Solid Tumors,https://www.clinicaltrials.gov/study/NCT03917043,Recruiting,"Advanced Solid Cancer, Non Small Cell Lung Can...",[{'Location Facility': 'Beijing Cancer Hospita...,"May 27, 2019",February 2025,18 Years||Adult|Older Adult,Eligibility includes patients with advanced tu...,The primary outcomes are to determine the maxi...,Secondary outcomes include measuring the maxim...,The intervention involves administering APG-24...,"{'Name': None, 'Organization': None, 'Email': ...","{""DRUGS"":[""APG-2449""],""BIOMARKERS"":[]}",APG-2449,
3,3,NCT05674422,GEMCAD-REVEAL STUDY - Circulating Tumor DNA as...,https://www.clinicaltrials.gov/study/NCT05674422,Recruiting,Patients With Locally Advanced Rectal Cancer,[{'Location Facility': 'Hospital Germans Trias...,"May 26, 2023",July 2026,18 Years||Adult|Older Adult,Eligible participants are legally capable indi...,The primary outcome measure is the detection o...,Secondary outcomes include the classification ...,The intervention involves total neoadjuvant th...,"{'Name': None, 'Organization': None, 'Email': ...","{""DRUGS"":[],""BIOMARKERS"":[]}",,
4,4,NCT06249854,Efficacy and Safety of Combination Therapy Wit...,https://www.clinicaltrials.gov/study/NCT06249854,Recruiting,Non-small Cell Lung Cancer,[{'Location Facility': 'Korea University Ansan...,"February 8, 2024","December 31, 2025",19 Years||Adult|Older Adult,Eligibility includes adults aged 19 or older w...,The primary outcome measure is Progression-Fre...,Secondary outcomes include Disease Control Rat...,The study involves two groups: one receiving P...,"{'Name': None, 'Organization': None, 'Email': ...","{\n""DRUGS"":[""Pembrolizumab"", ""Bojungikgitang""]...","Pembrolizumab, Bojungikgitang",PD-1 receptor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3831,3831,NCT05006131,Pancreatic Cancer Screening for At-risk Indivi...,https://www.clinicaltrials.gov/study/NCT05006131,Recruiting,"Pancreatic Cancer, Adult",[{'Location Facility': 'Beth Israel Deaconess ...,"July 10, 2020","July 10, 2026",18 Years|90 Years|Adult|Older Adult,Eligibility includes individuals indicated for...,The primary outcome is the proportion of scree...,Secondary outcomes include the proportion of s...,The intervention involves screening for pancre...,"{'Name': None, 'Organization': None, 'Email': ...","{""DRUGS"":[],""BIOMARKERS"":[""CAPS3""]}",,CAPS3
3832,3832,NCT04679064,Trial on NIraparib-TSR-042 (Dostarlimab) vs Ph...,htt

In [26]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [27]:
df

,NCT_NUMBER,STUDY_TITLE,STUDY_URL,STUDY_STATUS,CONDITIONS,LOCATIONS,START_DATE,COMPLETION_DATE,AGE,ELIGIBILITY_CRITERIA,PRIMARY_OUTCOMES,SECONDARY_OUTCOMES,INTERVENTIONS,POINT_OF_CONTACT,drugs & biomarker,DRUGS,BIOMARKERS
0,NCT05664971,JS004 Combined With Toripalimab and With Stand...,https://www.clinicaltrials.gov/study/NCT05664971,Recruiting,Advanced Lung Cancer,[{'Location Facility': 'Shanghai Chest Hospita...,"February 9, 2023","November 17, 2024",18 Years||Adult|Older Adult,Eligible participants include males or females...,The primary outcomes include the incidence of ...,Secondary outcomes encompass the duration of r...,The study involves multiple cohorts receiving ...,"{'Name': None, 'Organization': None, 'Email': ...","{""DRUGS"":[""JS004"",""JS001"",""docetaxel"",""pemetre...","JS004, JS001, docetaxel, pemetrexed, carboplat...",
1,NCT05902520,Adoptive Cell Therapy Using Cancer Specific CD...,https://www.clinicaltrials.gov/study/NCT05902520,Recruiting,"HNSCC, Melanoma, Gynecologic Cancer, Colorecta...",[{'Location Facility': 'Providence Portland Me...,"June 19, 2023","May 19, 2026",18 Years||Adult|Older Adult,Eligibility includes adults over 18 with histo...,The primary outcome is to assess the safety of...,Secondary outcomes include evaluating the effi...,The study involves two experimental arms. Both...,"{'Name': None, 'Organization': None, 'Email': ...","{\n""DRUGS"":[""IL-2""],\n""BIOMARKERS"":[""CD39"", ""C...",IL-2,"CD39, CD103, PD-1"
2,NCT03917043,APG-2449 in Patients With Advanced Solid Tumors,https://www.clinicaltrials.gov/study/NCT03917043,Recruiting,"Advanced Solid Cancer, Non Small Cell Lung Can...",[{'Location Facility': 'Beijing Cancer Hospita...,"May 27, 2019",February 2025,18 Years||Adult|Older Adult,Eligibility includes patients with advanced tu...,The primary outcomes are to determine the maxi...,Secondary outcomes include measuring the maxim...,The intervention involves administering APG-24...,"{'Name': None, 'Organization': None, 'Email': ...","{""DRUGS"":[""APG-2449""],""BIOMARKERS"":[]}",APG-2449,
3,NCT05674422,GEMCAD-REVEAL STUDY - Circulating Tumor DNA as...,https://www.clinicaltrials.gov/study/NCT05674422,Recruiting,Patients With Locally Advanced Rectal Cancer,[{'Location Facility': 'Hospital Germans Trias...,"May 26, 2023",July 2026,18 Years||Adult|Older Adult,Eligible participants are legally capable indi...,The primary outcome measure is the detection o...,Secondary outcomes include the classification ...,The intervention involves total neoadjuvant th...,"{'Name': None, 'Organization': None, 'Email': ...","{""DRUGS"":[],""BIOMARKERS"":[]}",,
4,NCT06249854,Efficacy and Safety of Combination Therapy Wit...,https://www.clinicaltrials.gov/study/NCT06249854,Recruiting,Non-small Cell Lung Cancer,[{'Location Facility': 'Korea University Ansan...,"February 8, 2024","December 31, 2025",19 Years||Adult|Older Adult,Eligibility includes adults aged 19 or older w...,The primary outcome measure is Progression-Fre...,Secondary outcomes include Disease Control Rat...,The study involves two groups: one receiving P...,"{'Name': None, 'Organization': None, 'Email': ...","{\n""DRUGS"":[""Pembrolizumab"", ""Bojungikgitang""]...","Pembrolizumab, Bojungikgitang",PD-1 receptor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3831,NCT05006131,Pancreatic Cancer Screening for At-risk Indivi...,https://www.clinicaltrials.gov/study/NCT05006131,Recruiting,"Pancreatic Cancer, Adult",[{'Location Facility': 'Beth Israel Deaconess ...,"July 10, 2020","July 10, 2026",18 Years|90 Years|Adult|Older Adult,Eligibility includes individuals indicated for...,The primary outcome is the proportion of scree...,Secondary outcomes include the proportion of s...,The intervention involves screening for pancre...,"{'Name': None, 'Organization': None, 'Email': ...","{""DRUGS"":[],""BIOMARKERS"":[""CAPS3""]}",,CAPS3
3832,NCT04679064,Trial on NIraparib-TSR-042 (Dostarlimab) vs Ph...,https://www.clinicaltrials.gov/study/N

In [28]:
df.to_csv('finalcttrial.csv',index=False)

In [27]:
df.columns

Index(['Unnamed: 0', 'NCT_NUMBER', 'STUDY_TITLE', 'STUDY_URL', 'STUDY_STATUS',
       'CONDITIONS', 'LOCATIONS', 'START_DATE', 'COMPLETION_DATE', 'AGE',
       'ELIGIBILITY_CRITERIA', 'PRIMARY_OUTCOMES', 'SECONDARY_OUTCOMES',
       'INTERVENTIONS', 'POINT_OF_CONTACT', 'drugs & biomarker', 'DRUGS',
       'BIOMARKERS'],
      dtype='object')

In [28]:
df['CONTENT'] = df[['STUDY_TITLE', 'ELIGIBILITY_CRITERIA','PRIMARY_OUTCOMES','SECONDARY_OUTCOMES','INTERVENTIONS']].apply(lambda x: ' '.join(x), axis=1)

In [29]:
df['CONTENT']

0       JS004 Combined With Toripalimab and With Stand...
1       Adoptive Cell Therapy Using Cancer Specific CD...
2       APG-2449 in Patients With Advanced Solid Tumor...
3       GEMCAD-REVEAL STUDY - Circulating Tumor DNA as...
4       Efficacy and Safety of Combination Therapy Wit...
                              ...                        
3831    Pancreatic Cancer Screening for At-risk Indivi...
3832    Trial on NIraparib-TSR-042 (Dostarlimab) vs Ph...
3833    DWIBS-MRI: An Adjunct to the Traditional Diagn...
3834    HER2(Human Epidermal Growth Factor Receptor 2)...
3835    Protective Effects of the Nutritional Suppleme...
Name: CONTENT, Length: 3836, dtype: object

In [30]:
df["CONTENT"][0:5].apply(get_drugs_biomarkers)

0    {"DRUGS":["JS004","Toripalimab","Standard Chem...
1    {"DRUGS":["IL-2","siRNA PD-1"],"BIOMARKERS":["...
2    {"DRUGS":["APG-2449"],"BIOMARKERS":["ALK","ROS...
3    {"DRUGS":[],"BIOMARKERS":["circulating tumor D...
4    {"DRUGS":["Bojungikki-tang","Pembrolizumab"],"...
Name: CONTENT, dtype: object

In [ ]:
df

In [8]:
df[""]

0       Recruiting
1       Recruiting
2       Recruiting
3       Recruiting
4       Recruiting
           ...    
3831    Recruiting
3832    Recruiting
3833    Recruiting
3834    Recruiting
3835    Recruiting
Name: STUDY_STATUS, Length: 3836, dtype: object